In [2]:
import pystatis

# Pystatis presentation

`pystatis` is a small Python library to conveniently wrap the different GENESIS web services (APIs) in a centralized and user-friendly manner.

It allows users to browse the different databases and download the desired tables from all supported databases in a convenient `pandas` `DataFrame` object, suited for further analysis.

## Setup

We won't cover the initial only-once setup here because the user has to enter their credentials for the supported databases (GENESIS, Regionalstatistik, Zensus). But there is a dedicated notebook [Setup](./00_Setup.ipynb) with examples and explanations.

## Main Use Cases

### Find

### Table

In [4]:
t = pystatis.Table(name="12111-01-01-5-B")

In [5]:
t.get_data()

PystatisConfigError: No active database set! Please run `set_db()`.